In [1]:
import pandas as pd
import numpy as np
from pydub import AudioSegment
from ast import literal_eval
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO, )
pd.set_option("display.max_colwidth", 100)
df = pd.read_csv("/home/peterr/macocu/task6_speech/8_results.csv")


def Getletterfromindex(num):
    import string
    num2alphadict = dict(zip(range(1, 27), string.ascii_lowercase))
    outval = ""
    numloops = (num-1) //26

    if numloops > 0:
        outval = outval + self.Getletterfromindex(numloops)

    remainder = num % 26
    if remainder > 0:
        outval = outval + num2alphadict[remainder]
    else:
        outval = outval + "z"
    return outval

df.head()

,path,sentence,duration,filesize,to_edit,cuts
0,/home/peterr/macocu/task6_speech/data/00019728.flac.wav,imamo odgovor na repliku,0.600,19278,False,0
1,/home/peterr/macocu/task6_speech/data/00013519.flac.wav,pa kako je to moguće,0.600,19278,False,0
2,/home/peterr/macocu/task6_speech/data/00021417.flac.wav,kaznenog zakona samo ide u prilog tezi da katalog kaznenih djela unatoč tome što je dopunjen i d...,0.629,20206,False,0
3,/home/peterr/macocu/task6_speech/data/00017866.flac.wav,ponovit ću još jedanput,0.700,22478,False,0
4,/home/peterr/macocu/task6_speech/data/00004197.flac.wav,evo to je moj odgovor,0.700,22478,False,0


In [2]:
file = "/home/peterr/macocu/task6_speech/data/00019728.flac.wav"

import os
os.path.dirname(file)

'/home/peterr/macocu/task6_speech/data'

Bookkeeping when cutting:

I could reconstruct the data from the file naming, but it is easier to keep the records of what is going on while doing it. I propose I keep the records of what files there are to be deleted, what new files are created and what their transcripts are.

In [3]:
c1 = ~df.cuts.isin((0, "0"))

files_to_delete = list()
new_files = list()

for row in df.loc[c1, :].iterrows():
    original_path = row[1]["path"]
    directory = os.path.dirname(original_path)
    filename = os.path.basename(original_path)

    transcript = row[1]["sentence"]
    cuts = literal_eval(row[1]["cuts"])

    if type(cuts) != list:
        logging.debug(
            f"File {filename} was not cut because the cuts look like this: {cuts=}")
        continue
    if len(cuts) <= 2:
        logging.debug(
            f"File {file} was not cut because the length of cuts is less or equal to 2. {cuts=}")
        continue
    audio = AudioSegment.from_file(original_path)
    for i, (start, end) in enumerate(zip(cuts, cuts[1:])):
        letter_suffix = Getletterfromindex(i+1)
        new_filename = filename.replace(
            ".flac.wav", f"{letter_suffix}.flac.wav")
        new_fullpath = os.path.join(directory, new_filename)
        chunk = audio[start:end]
        chunk.export(new_fullpath, format="wav")
        logging.debug(f"Exported file {new_fullpath}.")
        new_files.append(
            {
                "path": new_fullpath,
                "sentence_uncut": transcript,
                "original_file": original_path
            }
        )
        logging.debug(
            f"Appended new path and transcripts to the new_files list")

    files_to_delete.append(original_path)


In [5]:
new_df = pd.DataFrame(data=new_files)

new_df.to_csv("12_new_files.csv", index=False)